In [31]:
import os, json, gzip 
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

import random

import string
import nltk
nltk.download('stopwords')
nltk.download('wordnet')
nltk.download('punkt')
nltk.download('words')
nltk.download('averaged_perceptron_tagger')
nltk.download('omw-1.4')
from nltk.corpus import wordnet, stopwords

from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score
from sklearn.naive_bayes import MultinomialNB
from sklearn.linear_model import SGDClassifier
from sklearn.linear_model import LogisticRegression

[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/magalhaesm4/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     /Users/magalhaesm4/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     /Users/magalhaesm4/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package words to
[nltk_data]     /Users/magalhaesm4/nltk_data...
[nltk_data]   Package words is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /Users/magalhaesm4/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package omw-1.4 to
[nltk_data]     /Users/magalhaesm4/nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!


## Data Preparation

In [32]:
df = pd.read_csv('datasets/final_data.csv')
df.head()

,Unnamed: 0,name,city,cuisine_style,ranking,rating,price_range,number_of_reviews,reviews,restaurant_url,restaurant_id
0,0,McDonald's,Zurich,['American'],1540.0,2.5,$,11,"['Hangover', 'Most expensive McDonalds in the ...",https://www.tripadvisor.com/Restaurant_Review-...,d8796498
1,2,Yumi Hana,Zurich,"['Korean', 'Japanese']",1542.0,3.0,$$ - $$$,50,['What once was a 5 points Restaurant is clo.....,https://www.tripadvisor.com/Restaurant_Review-...,d2070374
2,3,Peking Garden,Zurich,"['Chinese', 'Asian']",1544.0,2.5,$,41,"['Just if no other choice!', 'Ok Chinese food']",https://www.tripadvisor.com/Restaurant_Review-...,d2005395
3,4,Long Huang,Zurich,"['Chinese', 'Vietnamese']",1545.0,2.5,$$ - $$$,14,"['Good', 'As the name implies, long wait and b...",https://www.tripadvisor.com/Restaurant_Review-...,d11963037
4,5,Restaurant Hallo,Zurich,"['European', 'Cafe']",1547.0,3.0,$$ - $$$,21,"['Really just ok', 'BRASILEIROS NÃO SÃO BEM-VI...",https://www.tripadvisor.com/Restaurant_Review-...,d7195130


In [33]:
df.drop(['Unnamed: 0','name', 'city', 'cuisine_style', 'ranking', 'price_range', 'number_of_reviews', 'restaurant_url'], axis=1)

,rating,reviews,restaurant_id
0,2.5,"['Hangover', 'Most expensive McDonalds in the ...",d8796498
1,3.0,['What once was a 5 points Restaurant is clo.....,d2070374
2,2.5,"['Just if no other choice!', 'Ok Chinese food']",d2005395
3,2.5,"['Good', 'As the name implies, long wait and b...",d11963037
4,3.0,"['Really just ok', 'BRASILEIROS NÃO SÃO BEM-VI...",d7195130
...,...,...,...
71590,2.5,"['Good food and nice experience', 'Disappointi...",d697907
71591,3.5,"['Good service', 'nice atmoshphere']",d1187556
71592,2.0,"['Worst New Year’s Eve experience', 'HORRIBLE']",d939089
71593,3.0,"['Horrible!', 'It was really horrible, I would...",d1551957


## Fundamental preprocessing tasks

In [34]:
stopwords_list = stopwords.words('english')

def ReviewProcessing(df):
  # remove non alphanumeric 
  df['review_cleaned'] = df.reviews.str.replace('[^a-zA-Z0-9 ]', '')
  # lowercase
  df.review_cleaned = df.review_cleaned.str.lower()
  # split into list
  df.review_cleaned = df.review_cleaned.str.split(' ')
  # remove stopwords
  df.review_cleaned = df.review_cleaned.apply(lambda x: [item for item in x if item not in stopwords_list])
  return df

## Lemmatization

In [35]:
def get_wordnet_pos(word):
  tag = nltk.pos_tag([word])[0][1][0].upper()
  tag_dict = {"J": wordnet.ADJ,
                "N": wordnet.NOUN,
                "V": wordnet.VERB,
                "R": wordnet.ADV}

  return tag_dict.get(tag, wordnet.NOUN)

lemmatizer = nltk.stem.WordNetLemmatizer()
def get_lemmatize(sent):
  return " ".join([lemmatizer.lemmatize(w, get_wordnet_pos(w)) for w in nltk.word_tokenize(sent)])

In [36]:
clean_data = ReviewProcessing(df)
clean_data.review_cleaned = clean_data.review_cleaned.apply(' '.join)
clean_data['review_cleaned_lemmatized'] = clean_data.review_cleaned.apply(get_lemmatize)
clean_data.head()

/var/folders/_r/29t8d54923b4pl12b1nwq531b09791/T/ipykernel_13294/1177820914.py:5: FutureWarning: The default value of regex will change from True to False in a future version.
  df['review_cleaned'] = df.reviews.str.replace('[^a-zA-Z0-9 ]', '')


,Unnamed: 0,name,city,cuisine_style,ranking,rating,price_range,number_of_reviews,reviews,restaurant_url,restaurant_id,review_cleaned,review_cleaned_lemmatized
0,0,McDonald's,Zurich,['American'],1540.0,2.5,$,11,"['Hangover', 'Most expensive McDonalds in the ...",https://www.tripadvisor.com/Restaurant_Review-...,d8796498,hangover expensive mcdonalds world,hangover expensive mcdonalds world
1,2,Yumi Hana,Zurich,"['Korean', 'Japanese']",1542.0,3.0,$$ - $$$,50,['What once was a 5 points Restaurant is clo.....,https://www.tripadvisor.com/Restaurant_Review-...,d2070374,5 points restaurant clo beware owner,5 point restaurant clo beware owner
2,3,Peking Garden,Zurich,"['Chinese', 'Asian']",1544.0,2.5,$,41,"['Just if no other choice!', 'Ok Chinese food']",https://www.tripadvisor.com/Restaurant_Review-...,d2005395,choice ok chinese food,choice ok chinese food
3,4,Long Huang,Zurich,"['Chinese', 'Vietnamese']",1545.0,2.5,$$ - $$$,14,"['Good', 'As the name implies, long wait and b...",https://www.tripadvisor.com/Restaurant_Review-...,d11963037,good name implies long wait bad ser,good name implies long wait bad ser
4,5,Restaurant Hallo,Zurich,"['European', 'Cafe']",1547.0,3.0,$$ - $$$,21,"['Really just ok', 'BRASILEIROS NÃO SÃO BEM-VI...",https://www.tripadvisor.com/Restaurant_Review-...,d7195130,really ok brasileiros bemvindos,really ok brasileiros bemvindos


### TF-IDF Weighting

In [37]:
x = clean_data['review_cleaned_lemmatized']
y = clean_data['rating']
y=y.astype(int)
X_train, X_test, y_train, y_test = train_test_split(x, y, test_size=0.2, stratify=y, random_state = 44)

In [38]:
from sklearn.naive_bayes import MultinomialNB

nb = Pipeline([('vectorize', CountVectorizer(ngram_range=(1, 2))),
               ('tfidf', TfidfTransformer()),
               ('clf', MultinomialNB()),
              ])

### Linear Support Vector Machine

In [39]:
from sklearn.linear_model import SGDClassifier

sgd = Pipeline([('vect', CountVectorizer(ngram_range=(1, 2))),
                ('tfidf', TfidfTransformer()),
                ('clf', SGDClassifier()),
               ])

### Logistic Regression Classifier

In [40]:
from sklearn.linear_model import LogisticRegression

logreg = Pipeline([('vect', CountVectorizer(ngram_range=(1, 2))),
                ('tfidf', TfidfTransformer()),
                ('clf', LogisticRegression(max_iter=500)),
               ])

In [41]:
# Naive Bayes
nb.fit(X_train, y_train)
y_pred_nb = nb.predict(X_test)
print("Naive Bayes:\n")
print(accuracy_score(y_test, y_pred_nb))
print(confusion_matrix(y_test, y_pred_nb))
print(classification_report(y_test, y_pred_nb))

# SGD Classifier
sgd.fit(X_train, y_train)
y_pred_sgd = sgd.predict(X_test)
print("SGD Classifier:\n")
print(accuracy_score(y_test, y_pred_sgd))
print(confusion_matrix(y_test, y_pred_sgd))
print(classification_report(y_test, y_pred_sgd))

# Logistic Regression
logreg.fit(X_train, y_train)
y_pred_log = logreg.predict(X_test)
print("Logistic regression:\n")
print(accuracy_score(y_test, y_pred_log))
print(confusion_matrix(y_test, y_pred_log))
print(classification_report(y_test, y_pred_log))

Naive Bayes:

0.6878273622459669
[[   0    0    1   28    0]
 [   0    0    0  293    0]
 [   0    0   14 3414    0]
 [   0    0    3 9835    0]
 [   0    0    0  731    0]]
              precision    recall  f1-score   support

           1       0.00      0.00      0.00        29
           2       0.00      0.00      0.00       293
           3       0.78      0.00      0.01      3428
           4       0.69      1.00      0.81      9838
           5       0.00      0.00      0.00       731

    accuracy                           0.69     14319
   macro avg       0.29      0.20      0.16     14319
weighted avg       0.66      0.69      0.56     14319



/Users/magalhaesm4/opt/anaconda3/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1334: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/Users/magalhaesm4/opt/anaconda3/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1334: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/Users/magalhaesm4/opt/anaconda3/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1334: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


SGD Classifier:

0.7029122145401215
[[   0    0    6   23    0]
 [   0    1   74  218    0]
 [   0    0  426 3002    0]
 [   0    1  199 9638    0]
 [   0    0    2  729    0]]
              precision    recall  f1-score   support

           1       0.00      0.00      0.00        29
           2       0.50      0.00      0.01       293
           3       0.60      0.12      0.21      3428
           4       0.71      0.98      0.82      9838
           5       0.00      0.00      0.00       731

    accuracy                           0.70     14319
   macro avg       0.36      0.22      0.21     14319
weighted avg       0.64      0.70      0.61     14319



/Users/magalhaesm4/opt/anaconda3/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1334: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/Users/magalhaesm4/opt/anaconda3/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1334: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/Users/magalhaesm4/opt/anaconda3/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1334: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


Logistic regression:

0.7091277323835463
[[   0    2   16   11    0]
 [   0    2  151  140    0]
 [   0    6  862 2558    2]
 [   0    5  544 9285    4]
 [   0    0    5  721    5]]
              precision    recall  f1-score   support

           1       0.00      0.00      0.00        29
           2       0.13      0.01      0.01       293
           3       0.55      0.25      0.34      3428
           4       0.73      0.94      0.82      9838
           5       0.45      0.01      0.01       731

    accuracy                           0.71     14319
   macro avg       0.37      0.24      0.24     14319
weighted avg       0.66      0.71      0.65     14319



/Users/magalhaesm4/opt/anaconda3/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1334: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/Users/magalhaesm4/opt/anaconda3/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1334: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/Users/magalhaesm4/opt/anaconda3/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1334: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [42]:
from sklearn.model_selection import GridSearchCV

grid=[{'clf__solver': ['lbfgs', 'sag', 'saga'],
       'clf__C': [0.01, 0.1, 1]}]
lr = GridSearchCV(logreg, param_grid = grid, cv = 5, scoring='accuracy', verbose = 1, n_jobs = -1)
best_model = lr.fit(X_train, y_train)

print(best_model.best_estimator_)
print(best_model.best_score_)

y_pred_grid = best_model.predict(X_test)
print(confusion_matrix(y_test, y_pred_grid))
print(classification_report(y_test, y_pred_grid))
print(accuracy_score(y_test, y_pred_grid))

Fitting 5 folds for each of 9 candidates, totalling 45 fits
Pipeline(steps=[('vect', CountVectorizer(ngram_range=(1, 2))),
                ('tfidf', TfidfTransformer()),
                ('clf', LogisticRegression(C=1, max_iter=500, solver='sag'))])
0.7096864628775705
[[   0    2   16   11    0]
 [   0    2  151  140    0]
 [   0    6  862 2558    2]
 [   0    5  544 9285    4]
 [   0    0    5  721    5]]
              precision    recall  f1-score   support

           1       0.00      0.00      0.00        29
           2       0.13      0.01      0.01       293
           3       0.55      0.25      0.34      3428
           4       0.73      0.94      0.82      9838
           5       0.45      0.01      0.01       731

    accuracy                           0.71     14319
   macro avg       0.37      0.24      0.24     14319
weighted avg       0.66      0.71      0.65     14319

0.7091277323835463


/Users/magalhaesm4/opt/anaconda3/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1334: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/Users/magalhaesm4/opt/anaconda3/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1334: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/Users/magalhaesm4/opt/anaconda3/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1334: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [43]:
review_train = random.sample(list(clean_data['review_cleaned_lemmatized']), 20)

In [44]:
for text in review_train:
    print(text)
    print(logreg.predict([text]))
    print("------")

excellent food service great loca great lunch pop brixton
[4]
------
lovely restaurant excellent food food immensed
[4]
------
great north african food madrid cozy great menu de degustacion ab fab st
[4]
------
great atmosphere late lunch
[4]
------
small friendly
[4]
------
nice vietnamese food nice vietnamese food
[4]
------
ahmazing turkish twist middle eastern
[4]
------
nice authentic food tasty koshari shame plastic
[4]
------
simple delicious hit miss
[4]
------
italian wife american wife stop glass wine dinner reservation ask two glass wine bartender look wife ask reply say apologeticallywe dont serve american hurtful experience rome love bar slightly tucked away reasonable drinkscocktail aperol spritz 5 eurosg 6 euro chill vibe great indie sound track friendly staff second night rome boyfriend idea go look trip advisor saw good recommendation place disappointed chill indie atmosphere relax sat outside watch people walk past night flew big fan indie vibe outside area suit perfe

In [45]:
sentiment = logreg.predict(clean_data['reviews'])
clean_data['sentiment'] = sentiment
clean_data.to_csv('datasets/reviews_sentiment_analysis.csv', index=False)